In [12]:
import os
from PIL import Image, ImageTk
import tkinter as tk
from collections import defaultdict

# === CONFIGURACIÓN ===
INPUT_DIR = "../original-images/"           # Carpeta donde están tus 18 imágenes
PATCH_SIZE = 64
POSITIVOS_DIR = "positivos"
NEGATIVOS_DIR = "negativos"
INFO_FILE = "info.dat"
BG_FILE = "bg.txt"

# === CREAR CARPETAS ===
os.makedirs(POSITIVOS_DIR, exist_ok=True)
os.makedirs(NEGATIVOS_DIR, exist_ok=True)

# === CARGAR TODAS LAS IMÁGENES ===
imagenes = sorted([f for f in os.listdir(INPUT_DIR) if f.lower().endswith(('.jpg', '.png'))])
img_index = 0
parches = []
coords = []
imagen_actual = None
img = None
img_width, img_height = 0, 0

# === DATOS DE SALIDA ===
positivos_dict = defaultdict(list)   # {ruta: [(x, y, w, h), ...]}
rutas_negativas = []                 # lista de imágenes negativas

# === FUNCIÓN PARA CARGAR UNA NUEVA IMAGEN ===
def cargar_siguiente_imagen():
    global img_index, imagen_actual, parches, coords, img, img_width, img_height
    if img_index >= len(imagenes):
        return False
    nombre_img = imagen_actual = imagenes[img_index]
    ruta_img = os.path.join(INPUT_DIR, nombre_img)
    img = Image.open(ruta_img)
    img_width, img_height = img.size

    parches.clear()
    coords.clear()

    for y in range(0, img_height, PATCH_SIZE):
        for x in range(0, img_width, PATCH_SIZE):
            parche = img.crop((x, y, x + PATCH_SIZE, y + PATCH_SIZE))
            parches.append(parche)
            coords.append((x, y))
    return True

# === TKINTER INTERFAZ ===
root = tk.Tk()
root.title("Clasificador de Parches")

indice_parche = 0

def mostrar_parche():
    global indice_parche, img_index
    if indice_parche >= len(parches):
        img_index += 1
        indice_parche = 0
        if cargar_siguiente_imagen():
            mostrar_parche()
        else:
            root.destroy()
            guardar_info()
        return
    imgtk = ImageTk.PhotoImage(parches[indice_parche].resize((250, 250)))
    panel.config(image=imgtk)
    panel.image = imgtk
    label_text.set(f"Imagen {img_index + 1}/{len(imagenes)} - Parcha {indice_parche + 1}/{len(parches)}")

def marcar(etiqueta):
    global indice_parche
    x, y = coords[indice_parche]
    w = h = PATCH_SIZE
    nombre_img = imagen_actual
    ruta_original = os.path.join(INPUT_DIR, nombre_img)

    if etiqueta == "positivo":
        ruta_pos = os.path.join(POSITIVOS_DIR, nombre_img)
        positivos_dict[ruta_pos].append((x, y, w, h))
        if not os.path.exists(ruta_pos):
            img.save(ruta_pos)

    elif etiqueta == "negativo":
        nombre_parche = f"parche_{nombre_img[:-4]}_{x}_{y}.jpg"
        ruta_parche = os.path.join(NEGATIVOS_DIR, nombre_parche)
        parches[indice_parche].save(ruta_parche)
        rutas_negativas.append(ruta_parche)

    indice_parche += 1
    mostrar_parche()

def guardar_info():
    # Guardar info.dat
    with open(INFO_FILE, "w") as f:
        for ruta, coords_list in positivos_dict.items():
            linea = f"{ruta} {len(coords_list)}"
            for (x, y, w, h) in coords_list:
                linea += f" {x} {y} {w} {h}"
            f.write(linea + "\n")
    print("✅ info.dat generado.")

    # Guardar bg.txt
    with open(BG_FILE, "w") as f:
        for ruta in rutas_negativas:
            f.write(ruta + "\n")
    print("✅ bg.txt generado.")

# === WIDGETS INTERFAZ ===
panel = tk.Label(root)
panel.pack()

label_text = tk.StringVar()
label = tk.Label(root, textvariable=label_text)
label.pack()

boton_positivo = tk.Button(root, text="✅ Éxito", command=lambda: marcar("positivo"))
boton_positivo.pack(side="left", padx=10, pady=10)

boton_negativo = tk.Button(root, text="❌ No éxito", command=lambda: marcar("negativo"))
boton_negativo.pack(side="right", padx=10, pady=10)

# === INICIO ===
if cargar_siguiente_imagen():
    mostrar_parche()
    root.mainloop()
else:
    print("❌ No hay imágenes en la carpeta.")


2025-05-16 08:09:13.750 python[71733:30231959] _mthid_copyDeviceInfo(216172782148386816) failed


In [10]:
from PIL import Image

# Parámetros
ruta_imagen = "../original-images/1.jpg"  # Cambia por el nombre de tu imagen
patch_size = 256  # Tamaño de parche, por ejemplo 32x32

# Abrir imagen y obtener dimensiones
img = Image.open(ruta_imagen)
width, height = img.size

# Calcular cantidad de parches en cada dirección
num_patches_x = width // patch_size
num_patches_y = height // patch_size
total_patches = num_patches_x * num_patches_y

# Mostrar resultado
print(f"Tamaño de la imagen: {width} x {height}")
print(f"Patch size: {patch_size} x {patch_size}")
print(f"Cantidad de parches horizontales: {num_patches_x}")
print(f"Cantidad de parches verticales: {num_patches_y}")
print(f"Total de parches a revisar: {total_patches}")

Tamaño de la imagen: 2048 x 1251
Patch size: 256 x 256
Cantidad de parches horizontales: 8
Cantidad de parches verticales: 4
Total de parches a revisar: 32


In [1]:
import os
from PIL import Image, ImageTk
import tkinter as tk
from collections import defaultdict

# === CONFIGURACIÓN ===
INPUT_DIR = "../original-images/"
PATCH_SIZE = 64
POSITIVOS_DIR = "positivos"
NEGATIVOS_DIR = "negativos"
INFO_FILE = "info.dat"
BG_FILE = "bg.txt"
POS_FILE = "pos.txt"
NEG_FILE = "neg.txt"

# === CREAR CARPETAS ===
os.makedirs(POSITIVOS_DIR, exist_ok=True)
os.makedirs(NEGATIVOS_DIR, exist_ok=True)

# === CARGAR TODAS LAS IMÁGENES DISPONIBLES ===
imagenes = sorted([f for f in os.listdir(INPUT_DIR) if f.lower().endswith(('.jpg', '.png'))])

# === DATOS DE SALIDA ===
positivos_dict = defaultdict(list)
rutas_negativas = []

# === INTERFAZ PARA SELECCIÓN DE IMAGEN ===
def seleccionar_imagen():
    selector = tk.Tk()
    selector.title("Selecciona la imagen a etiquetar")

    lista = tk.Listbox(selector, width=50)
    for idx, img in enumerate(imagenes):
        lista.insert(idx, img)
    lista.pack()

    def cargar_seleccion():
        global imagen_seleccionada
        sel = lista.curselection()
        if sel:
            imagen_seleccionada = imagenes[sel[0]]
            selector.destroy()
            iniciar_etiquetado(imagen_seleccionada)

    boton = tk.Button(selector, text="Etiquetar imagen seleccionada", command=cargar_seleccion)
    boton.pack()
    selector.mainloop()

# === ETIQUETADO ===
def iniciar_etiquetado(nombre_img):
    global imagen_actual, parches, coords, img, img_width, img_height, indice_parche

    ruta_img = os.path.join(INPUT_DIR, nombre_img)
    imagen_actual = nombre_img
    img = Image.open(ruta_img)
    img_width, img_height = img.size

    parches.clear()
    coords.clear()

    for y in range(0, img_height, PATCH_SIZE):
        for x in range(0, img_width, PATCH_SIZE):
            parche = img.crop((x, y, x + PATCH_SIZE, y + PATCH_SIZE))
            parches.append(parche)
            coords.append((x, y))

    mostrar_parche()

def mostrar_parche():
    global indice_parche
    if indice_parche >= len(parches):
        guardar_info()
        root.destroy()
        return
    imgtk = ImageTk.PhotoImage(parches[indice_parche].resize((250, 250)))
    panel.config(image=imgtk)
    panel.image = imgtk
    label_text.set(f"{imagen_actual} - Parche {indice_parche + 1}/{len(parches)}")

def marcar(etiqueta):
    global indice_parche
    x, y = coords[indice_parche]
    w = h = PATCH_SIZE
    ruta_original = os.path.join(INPUT_DIR, imagen_actual)

    if etiqueta == "positivo":
        ruta_pos = os.path.join(POSITIVOS_DIR, imagen_actual)
        positivos_dict[ruta_pos].append((x, y, w, h))
        if not os.path.exists(ruta_pos):
            img.save(ruta_pos)
    elif etiqueta == "negativo":
        nombre_parche = f"parche_{imagen_actual[:-4]}_{x}_{y}.jpg"
        ruta_parche = os.path.join(NEGATIVOS_DIR, nombre_parche)
        parches[indice_parche].save(ruta_parche)
        rutas_negativas.append(ruta_parche)

    indice_parche += 1
    mostrar_parche()

def guardar_info():
    # Guardar info.dat
    with open(INFO_FILE, "w") as f:
        for ruta, coords_list in positivos_dict.items():
            linea = f"{ruta} {len(coords_list)}"
            for (x, y, w, h) in coords_list:
                linea += f" {x} {y} {w} {h}"
            f.write(linea + "\n")
    print("✅ info.dat generado.")

    # Guardar bg.txt
    with open(BG_FILE, "w") as f:
        for ruta in rutas_negativas:
            f.write(ruta + "\n")
    print("✅ bg.txt generado.")

    # Guardar pos.txt y neg.txt
    with open(POS_FILE, "w") as f:
        for ruta, coords_list in positivos_dict.items():
            rel_path = os.path.relpath(ruta, '.')  # o cambia a 'img/...' si prefieres
            linea = f"{rel_path} {len(coords_list)}"
            for (x, y, w, h) in coords_list:
                linea += f" {x} {y} {w} {h}"
            f.write(linea + "\n")

    with open(NEG_FILE, "w") as f:
        for ruta in rutas_negativas:
            rel_path = os.path.relpath(ruta, '.')
            f.write(rel_path + "\n")

    print("✅ pos.txt y neg.txt generados.")

# === INICIO DE INTERFAZ DE ETIQUETADO ===
imagen_actual = ""
parches = []
coords = []
indice_parche = 0

root = tk.Tk()
root.title("Clasificador de Parches")

panel = tk.Label(root)
panel.pack()

label_text = tk.StringVar()
label = tk.Label(root, textvariable=label_text)
label.pack()

boton_positivo = tk.Button(root, text="✅ Éxito", command=lambda: marcar("positivo"))
boton_positivo.pack(side="left", padx=10, pady=10)

boton_negativo = tk.Button(root, text="❌ No éxito", command=lambda: marcar("negativo"))
boton_negativo.pack(side="right", padx=10, pady=10)

# Lanzar selección de imagen
seleccionar_imagen()
root.mainloop()


: 

In [1]:
import os
from PIL import Image, ImageTk
import tkinter as tk
from collections import defaultdict

# === CONFIGURACIÓN ===
INPUT_DIR = "../original-images/"
IMAGEN_A_ETIQUETAR = "1.jpg"  # Cambia esto por la imagen que quieras
PATCH_SIZE = 64
POSITIVOS_DIR = "positivos"
NEGATIVOS_DIR = "negativos"
INFO_FILE = "info.dat"
BG_FILE = "bg.txt"
POS_FILE = "pos.txt"
NEG_FILE = "neg.txt"

# === CREAR CARPETAS ===
os.makedirs(POSITIVOS_DIR, exist_ok=True)
os.makedirs(NEGATIVOS_DIR, exist_ok=True)

# === DATOS DE SALIDA ===
positivos_dict = defaultdict(list)
rutas_negativas = []

# === FUNCIONES DE ETIQUETADO ===
def iniciar_etiquetado(nombre_img):
    global imagen_actual, parches, coords, img, img_width, img_height, indice_parche

    ruta_img = os.path.join(INPUT_DIR, nombre_img)
    if not os.path.exists(ruta_img):
        print(f"❌ Imagen no encontrada: {ruta_img}")
        return

    imagen_actual = nombre_img
    img = Image.open(ruta_img)
    img_width, img_height = img.size

    parches.clear()
    coords.clear()

    for y in range(0, img_height, PATCH_SIZE):
        for x in range(0, img_width, PATCH_SIZE):
            parche = img.crop((x, y, x + PATCH_SIZE, y + PATCH_SIZE))
            parches.append(parche)
            coords.append((x, y))

    mostrar_parche()

def mostrar_parche():
    global indice_parche
    if indice_parche >= len(parches):
        guardar_info()
        root.destroy()
        return
    imgtk = ImageTk.PhotoImage(parches[indice_parche].resize((250, 250)))
    panel.config(image=imgtk)
    panel.image = imgtk
    label_text.set(f"{imagen_actual} - Parche {indice_parche + 1}/{len(parches)}")

def marcar(etiqueta):
    global indice_parche
    x, y = coords[indice_parche]
    w = h = PATCH_SIZE
    ruta_original = os.path.join(INPUT_DIR, imagen_actual)

    if etiqueta == "positivo":
        ruta_pos = os.path.join(POSITIVOS_DIR, imagen_actual)
        positivos_dict[ruta_pos].append((x, y, w, h))
        if not os.path.exists(ruta_pos):
            img.save(ruta_pos)
    elif etiqueta == "negativo":
        nombre_parche = f"parche_{imagen_actual[:-4]}_{x}_{y}.jpg"
        ruta_parche = os.path.join(NEGATIVOS_DIR, nombre_parche)
        parches[indice_parche].save(ruta_parche)
        rutas_negativas.append(ruta_parche)

    indice_parche += 1
    mostrar_parche()

def guardar_info():
    # Guardar info.dat
    with open(INFO_FILE, "w") as f:
        for ruta, coords_list in positivos_dict.items():
            linea = f"{ruta} {len(coords_list)}"
            for (x, y, w, h) in coords_list:
                linea += f" {x} {y} {w} {h}"
            f.write(linea + "\n")
    print("✅ info.dat generado.")

    # Guardar bg.txt
    with open(BG_FILE, "w") as f:
        for ruta in rutas_negativas:
            f.write(ruta + "\n")
    print("✅ bg.txt generado.")

    # Guardar pos.txt y neg.txt
    with open(POS_FILE, "w") as f:
        for ruta, coords_list in positivos_dict.items():
            rel_path = os.path.relpath(ruta, '.')
            linea = f"{rel_path} {len(coords_list)}"
            for (x, y, w, h) in coords_list:
                linea += f" {x} {y} {w} {h}"
            f.write(linea + "\n")

    with open(NEG_FILE, "w") as f:
        for ruta in rutas_negativas:
            rel_path = os.path.relpath(ruta, '.')
            f.write(rel_path + "\n")

    print("✅ pos.txt y neg.txt generados.")

# === VARIABLES GLOBALES ===
imagen_actual = ""
parches = []
coords = []
indice_parche = 0

# === INICIAR INTERFAZ ===
root = tk.Tk()
root.title("Clasificador de Parches")

panel = tk.Label(root)
panel.pack()

label_text = tk.StringVar()
label = tk.Label(root, textvariable=label_text)
label.pack()

boton_positivo = tk.Button(root, text="✅ Éxito", command=lambda: marcar("positivo"))
boton_positivo.pack(side="left", padx=10, pady=10)

boton_negativo = tk.Button(root, text="❌ No éxito", command=lambda: marcar("negativo"))
boton_negativo.pack(side="right", padx=10, pady=10)

# === EMPEZAR CON IMAGEN SELECCIONADA ===
iniciar_etiquetado(IMAGEN_A_ETIQUETAR)
root.mainloop()
root.destroy()
root.quit()

TclError: can't invoke "destroy" command: application has been destroyed